In [218]:
import numba

import datetime
import pandas as pd
import numpy as np
#import pandas_profiling
import sys
import os

import time
from datetime import date
from datetime import timedelta
from datetime import datetime

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import neighbors
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.neighbors import KNeighborsRegressor
from scipy import stats
import sklearn.metrics
from random import randrange
from itertools import chain
import random

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(font_scale=2)
pd.options.display.float_format = '{:,.9f}'.format

pd.set_option('display.max_columns', 600)
pd.set_option('display.max_rows', 600)

In [219]:
df = pd.read_csv('HORSE_RACING_TABLE.csv')

C:\Users\ivan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (546) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Dummy columns for SEX_TYPE (C/F/G/H/M/R), Result (Winner/Loser/Removed)

In [220]:
df = pd.concat([df,pd.get_dummies(df.SEX_TYPE)], axis = 1)
df = pd.concat([df,pd.get_dummies(df.HORSE_RESULT)], axis = 1)

## Reward Column

In [221]:
df['REWARD'] = (df.WINNER * df.BACKPRICE)-1

## Corrected BACKPRICE (correcting for overround)

In [222]:
market_list = df.groupby('MARKETID').count().index.tolist()
    
s1 = pd.DataFrame({'BACKPRICE' : []})

for m_ID in market_list:

    over_round = np.sum(1/(df[df.MARKETID == m_ID].BACKPRICE))

    s2 = df[df.MARKETID == m_ID].BACKPRICE*over_round
    s2 = s2.to_frame()
    s1 = s1.append(s2, ignore_index = False)
#s1.columns = ['CORRECTED_BACKPRICE']
s1 = s1.rename(columns={"BACKPRICE": "CORRECTED_BACKPRICE"})

df = pd.merge(df, s1, right_index=True, left_index=True)

## Induced Odds (from BACKPRICE)

In [223]:
df['INDUCED_ODDS']           = 1/df.BACKPRICE
df['CORRECTED_INDUCED_ODDS'] = 1/df.CORRECTED_BACKPRICE
df['BACK_LAY_DIFF']          = df.BACKPRICE - df.LAYPRICE  

## Fraction Amount Backed on each horse

In [8]:
market_list = df.groupby('MARKETID').count().index.tolist()

time_out_list = ['2_SECS_OUT',
                 '1_MIN_OUT',
                 '5_MINS_OUT',
                 '10_MINS_OUT',
                 '20_MINS_OUT',
                 '30_MINS_OUT',
                 '40_MINS_OUT',
                 '45_MINS_OUT',
                 '60_MINS_OUT']

for time_out in time_out_list:
    
    s1 = pd.DataFrame({'AMNT_BACKED_' + time_out : []})

    for m_ID in market_list[:200]:

        total_backed = np.sum(df[df.MARKETID == m_ID]['AMT_BACKED_' + time_out])

        s2 = df[df.MARKETID == m_ID]['AMT_BACKED_' + time_out]/total_backed
        s2 = s2.to_frame()
        s1 = s1.append(s2, ignore_index = False)


    s1 = s1.rename(columns={'AMT_BACKED_' + time_out: "FRXN_AMNT_BACKED_" + time_out})

    df = pd.merge(df, s1, right_index=True, left_index=True)

C:\Users\ivan\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


## Change in Price, Amount Backed at each time step

In [18]:
for i in range(len(time_out_list)-1):
    
    #df['DEL_' + time_out_list[i] + '_' + time_out_list[i+1]] = df['AMT_BACKED_' + time_out_list[i]] - \
    #                                                           df['AMT_BACKED_' + time_out_list[i+1]]
    
    #df['DEL_' + time_out_list[i] + '_' + time_out_list[i+1]] = df['PRICE_' + time_out_list[i]] - \
    #                                                           df['PRICE_' + time_out_list[i+1]]
    
    #df['DEL_' + time_out_list[i] + '_' + time_out_list[i+1]] = df['FRXN_AMNT_BACKED_' + time_out_list[i]] - \
    #                                                           df['FRXN_AMNT_BACKED_' + time_out_list[i+1]]
    
    df['INDUCED_ODDS_' + time_out_list[i]] = 1/df['PRICE_' + time_out_list[i]]
    

    
    
#df['DEL_START_END_AMNT_BACKED']      = df['AMT_BACKED_2_SECS_OUT']       - df['AMT_BACKED_60_MINS_OUT']
#df['DEL_START_END_PRICE']            = df['PRICE_2_SECS_OUT']            - df['PRICE_60_MINS_OUT']
#df['DEL_START_END_FRXN_AMNT_BACKED'] = df['FRXN_AMNT_BACKED_2_SECS_OUT'] - df['FRXN_AMNT_BACKED_60_MINS_OUT']
    

## Normalised Features 
### Group by race, normalise to maximum value.

## Normalising to the max value (most likely horse) keeps all the values linear and between 0 & 1.

In [20]:
market_list = df.groupby('MARKETID').count().index.tolist()

feat_list = ['AMT_BACKED_1_MIN_OUT',
             'AMT_BACKED_10_MINS_OUT',
             'AMT_BACKED_20_MINS_OUT',
             'AMT_BACKED_2_SECS_OUT',
             'AMT_BACKED_30_MINS_OUT',
             'AMT_BACKED_40_MINS_OUT',
             'AMT_BACKED_45_MINS_OUT',
             'AMT_BACKED_5_MINS_OUT',
             'AMT_BACKED_60_MINS_OUT',
             'PRICE_10_MINS_OUT',
             'PRICE_1_MIN_OUT',
             'PRICE_20_MINS_OUT',
             'PRICE_2_SECS_OUT',
             'PRICE_30_MINS_OUT',
             'PRICE_40_MINS_OUT',
             'PRICE_45_MINS_OUT',
             'PRICE_5_MINS_OUT',
             'PRICE_60_MINS_OUT',
             'BACKPRICE',
             'AGE',
             'WEIGHT_VALUE',
             'JOCKEY_CLAIM',
             'PROPENSITY_RANKING',
             'HANDICAP',
             'INDUCED_ODDS',
             'CORRECTED_INDUCED_ODDS']

for feat in feat_list:
    
    s1 = pd.DataFrame({feat : []})
    
    for m_ID in market_list:

        max_value = df[df.MARKETID == m_ID][feat].max()
                
        s2 = df[df.MARKETID == m_ID][feat]/max_value
        s2 = s2.to_frame()
        s1 = s1.append(s2, ignore_index = False)
    s1.columns = [feat + '_NORM']


    df = pd.merge(df, s1, right_index=True, left_index=True)

In [ ]:
now = time.time()

market_list = df.groupby('MARKETID').count().index.tolist()
feat = 'CORRECTED_INDUCED_ODDS'
s1 = pd.DataFrame({feat : []})

for m_ID in market_list:

    max_value = df[df.MARKETID == m_ID][feat].max()

    s2 = df[df.MARKETID == m_ID][feat]/max_value
    s2 = s2.to_frame()
    s1 = s1.append(s2, ignore_index = False)
s1.columns = [feat + '_NORM']


df = pd.merge(df, s1, right_index=True, left_index=True)

elapsed_time = int(time.time() - now)
print elapsed_time

## Adding columns which give the normalised corrected induced odds for the other horses

In [226]:
now = time.time()

market_list = df.groupby('MARKETID').count().index.tolist()
feat = 'CORRECTED_INDUCED_ODDS'
s1 = pd.DataFrame({feat : []})

for m_ID in market_list:
    
    max_value = df[df.MARKETID == m_ID][feat].max()
    other = df[df.MARKETID == m_ID][feat]/max_value
    s2 = df[df.MARKETID == m_ID][feat]/max_value

    s2 = s2.to_frame()
    for i in range(30):
        s2[feat + 'NORM_Other_Horse_' + str(i)] = 0

    for i in range(len(other)):

        index = other.index[i]

        for j in range(i+1, 1+i+len(other)):
            #j=j+1
            s2.loc[index,feat + 'NORM_Other_Horse_' + str(j % len(other))] = other.values[(j+i) % (len(other))]
    
    s1 = s1.append(s2, ignore_index = False)
    
df = pd.merge(df, s1, right_index=True, left_index=True)

In [229]:
elapsed_time = int(time.time() - now)
print (elapsed_time)

2350


## Horse Form as separate columns. (Race 1 is most recent)

### Link explaining horse form. https://help.racingpost.com/hc/en-us/articles/115001699689-Abbreviations-on-the-racecard

In [230]:
df.HORSE_FORM = df.HORSE_FORM.fillna('0')

def form_split(form):
    s = ''

    for i in range(6):
        try:
            s = s + list(form)[i] + ' '
        except:
            s = s + '0 '
    return s

df['HORSE_FORM'] = df.apply(lambda row: form_split(row['HORSE_FORM']), axis=1)


df[['Race_6','Race_5','Race_4','Race_3','Race_2','Race_1']] = df.HORSE_FORM.str.split(" ",n = 5, expand=True,)


#df.groupby('Race_6').count().index

df["Race_6"]= df["Race_6"].replace('B', 0)
df["Race_6"]= df["Race_6"].replace('C', 0)
df["Race_6"]= df["Race_6"].replace('F', 0)
df["Race_6"]= df["Race_6"].replace('P', 0)
df["Race_6"]= df["Race_6"].replace('R', 0)
df["Race_6"]= df["Race_6"].replace('S', 0)
df["Race_6"]= df["Race_6"].replace('U', 0)
df["Race_6"]= df["Race_6"].replace('-', 0)
df["Race_6"]= df["Race_6"].replace('/', 0)

df["Race_5"]= df["Race_5"].replace('B', 0)
df["Race_5"]= df["Race_5"].replace('C', 0)
df["Race_5"]= df["Race_5"].replace('F', 0)
df["Race_5"]= df["Race_5"].replace('P', 0)
df["Race_5"]= df["Race_5"].replace('R', 0)
df["Race_5"]= df["Race_5"].replace('S', 0)
df["Race_5"]= df["Race_5"].replace('U', 0)
df["Race_5"]= df["Race_5"].replace('-', 0)
df["Race_5"]= df["Race_5"].replace('/', 0)

df["Race_4"]= df["Race_4"].replace('B', 0)
df["Race_4"]= df["Race_4"].replace('C', 0)
df["Race_4"]= df["Race_4"].replace('F', 0)
df["Race_4"]= df["Race_4"].replace('P', 0)
df["Race_4"]= df["Race_4"].replace('R', 0)
df["Race_4"]= df["Race_4"].replace('S', 0)
df["Race_4"]= df["Race_4"].replace('U', 0)
df["Race_4"]= df["Race_4"].replace('-', 0)
df["Race_4"]= df["Race_4"].replace('/', 0)

df["Race_3"]= df["Race_3"].replace('B', 0)
df["Race_3"]= df["Race_3"].replace('C', 0)
df["Race_3"]= df["Race_3"].replace('F', 0)
df["Race_3"]= df["Race_3"].replace('P', 0)
df["Race_3"]= df["Race_3"].replace('R', 0)
df["Race_3"]= df["Race_3"].replace('S', 0)
df["Race_3"]= df["Race_3"].replace('U', 0)
df["Race_3"]= df["Race_3"].replace('-', 0)
df["Race_3"]= df["Race_3"].replace('/', 0)

df["Race_2"]= df["Race_2"].replace('B', 0)
df["Race_2"]= df["Race_2"].replace('C', 0)
df["Race_2"]= df["Race_2"].replace('F', 0)
df["Race_2"]= df["Race_2"].replace('P', 0)
df["Race_2"]= df["Race_2"].replace('R', 0)
df["Race_2"]= df["Race_2"].replace('S', 0)
df["Race_2"]= df["Race_2"].replace('U', 0)
df["Race_2"]= df["Race_2"].replace('-', 0)
df["Race_2"]= df["Race_2"].replace('/', 0)

df["Race_1"]= df["Race_1"].replace('B ', 0)
df["Race_1"]= df["Race_1"].replace('C ', 0)
df["Race_1"]= df["Race_1"].replace('F ', 0)
df["Race_1"]= df["Race_1"].replace('P ', 0)
df["Race_1"]= df["Race_1"].replace('R ', 0)
df["Race_1"]= df["Race_1"].replace('S ', 0)
df["Race_1"]= df["Race_1"].replace('U ', 0)
df["Race_1"]= df["Race_1"].replace('- ', 0)
df["Race_1"]= df["Race_1"].replace('/ ', 0)

df.Race_6 = pd.to_numeric(df.Race_6)
df.Race_5 = pd.to_numeric(df.Race_5)
df.Race_4 = pd.to_numeric(df.Race_4)
df.Race_3 = pd.to_numeric(df.Race_3)
df.Race_2 = pd.to_numeric(df.Race_2)
df.Race_1 = pd.to_numeric(df.Race_1)

## Replace inf and NaNs with 0

In [231]:
cols = sorted(df.columns.tolist())

df = df.replace([np.inf, -np.inf], np.nan)

for p in cols:
    df[p].fillna(0,inplace=True)

# Write the Modified dataset

In [232]:
df.to_csv('HORSE_DATA_MOD.csv')

In [ ]:
1+1

# Check the columns

In [233]:
df = pd.read_csv('HORSE_DATA_MOD.csv')

sorted(df.columns.tolist())

C:\Users\ivan\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (547) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


['ADJUSTED_RATING',
 'AGE',
 'AMT_BACKED_10_MINS_OUT',
 'AMT_BACKED_1_MIN_OUT',
 'AMT_BACKED_20_MINS_OUT',
 'AMT_BACKED_2_SECS_OUT',
 'AMT_BACKED_30_MINS_OUT',
 'AMT_BACKED_40_MINS_OUT',
 'AMT_BACKED_45_MINS_OUT',
 'AMT_BACKED_5_MINS_OUT',
 'AMT_BACKED_60_MINS_OUT',
 'BACKPRICE',
 'BACK_LAY_DIFF',
 'BRED',
 'C',
 'CLOTH_NUMBER',
 'CLOTH_NUMBER_ALPHA',
 'COLOURS_DESCRIPTION',
 'COLOUR_TYPE',
 'CORRECTED_BACKPRICE',
 'CORRECTED_INDUCED_ODDS_NORM',
 'CORRECTED_INDUCED_ODDS_Other_Horse_0',
 'CORRECTED_INDUCED_ODDS_Other_Horse_1',
 'CORRECTED_INDUCED_ODDS_Other_Horse_10',
 'CORRECTED_INDUCED_ODDS_Other_Horse_11',
 'CORRECTED_INDUCED_ODDS_Other_Horse_12',
 'CORRECTED_INDUCED_ODDS_Other_Horse_13',
 'CORRECTED_INDUCED_ODDS_Other_Horse_14',
 'CORRECTED_INDUCED_ODDS_Other_Horse_15',
 'CORRECTED_INDUCED_ODDS_Other_Horse_16',
 'CORRECTED_INDUCED_ODDS_Other_Horse_17',
 'CORRECTED_INDUCED_ODDS_Other_Horse_18',
 'CORRECTED_INDUCED_ODDS_Other_Horse_19',
 'CORRECTED_INDUCED_ODDS_Other_Horse_2',
 'CORRE

## Trainer/Jockey/Owner Ranking/Score Elo points....

# Add in the Final RF (or whatever) model and add Pred Prob as column. This is for the  RLearning part where it decides what's a good bet.

# Remove the SC columns

In [26]:
df.iloc[:, 111:544].head()

,SC_1_JOCKEY,SC_2_PCT_PRICE_CH_1M_TO_2S,SC_3_PCT_BACKED_2_SECS_OUT,SC_4_PCT_CH_AMT_60M_TO_2S,SC_5_PCT_CH_AMT_45M_TO_2S,SC_6_PCT_CH_AMT_40M_TO_2S,SC_7_PCT_CH_AMT_30M_TO_2S,SC_8_PCT_CH_AMT_20M_TO_2S,SC_9_PCT_CH_AMT_10M_TO_2S,SC_10_PCT_CH_AMT_5M_TO_2S,SC_11_PCT_CH_AMT_1M_TO_2S,SC_12_RACE_TRACK,SC_13_RACE_NAME,SC_14_DISTANCE,SC_15_RACE_TYPE,SC_16_DAYS_SINCE_LAST_RUN,SC_17_SEX_TYPE,SC_18_AGE,SC_19_TRAINER,SC_20_OWNER,SC_21_LAST_RACE,SC_22_LAST_TWO_RACES,SC_23_WEIGHT_VALUE,SC_24_JOCKEY_CLAIM,SC_25_WEARING,SC_26_OFFICIAL_RATING,SC_27_SIRE_NAME,SC_28_SIRE_YEAR_BORN,SC_29_SIRE_BRED,SC_30_DAM_NAME,SC_31_DAM_BRED,SC_32_DAM_YEAR_BORN,SC_33_DAMSIRE_NAME,SC_34_DAMSIRE_YEAR_BORN,SC_35_DAMSIRE_BRED,SC_36_STALL_DRAW,SC_37_ADJUSTED_RATING,SC_38_BRED,SC_39_OWNER_TRAINER,SC_40_JOCKEY_OWNER,SC_41_TRAINER_RACE_TRACK,SC_42_JOCKEY_RACE_TRACK,SC_43_OWNER_RACE_TRACK,SC_44_OWNER_RACE_TYPE,SC_45_TRAINER_RACE_TYPE,SC_46_JOCKEY_RACE_TYPE,SC_47_JOCKEY_PCT_BACKED_2S,SC_48_OWNER_PCT_BACKED_2S,SC_49_TRAINER_PCT_BACKED_2S,SC_50_JKY_PCT_CH_AMT_1M_TO_2S,SC_51_OWNR_PCT_CH_AMT_1M_TO_2S,SC_52_TRNR_PCT_CH_AMT_1M_TO_2S,SC_53_JOCKEY_DISTANCE,SC_54_OWNER_DISTANCE,SC_55_TRAINER_DISTANCE,SC_56_TRAINER_JOCKEY_DISTANCE,SC_57_TRAINER_JOCKEY,SC_58_TRAINER_JOCKEY_OWNER,SC_59_2ND_LAST_RACE,SC_60_TR_JK_OWN_RT,SC_61_SELECTIONID,SC_62_SELECTIONID_DISTANCE,SC_63_SELECTIONID_DIST_JKY,SC_64_TRAINER_NAME_AGE,SC_65_TRAINER_SIRE,SC_66_TRAINER_SIRE_DAM,SC_67_OWNER_SIRE,SC_68_OWNER_SIRE_DAM,SC_69_TRAINER_LAST_2_RACES,SC_70_OWNER_LAST_2_RACES,SC_1_WLR,SC_1_NUM_SAMPLES,SC_2_WLR,SC_2_NUM_SAMPLES,SC_3_WLR,SC_3_NUM_SAMPLES,SC_4_WLR,SC_4_NUM_SAMPLES,SC_5_WLR,SC_5_NUM_SAMPLES,SC_6_WLR,SC_6_NUM_SAMPLES,SC_7_WLR,SC_7_NUM_SAMPLES,SC_8_WLR,SC_8_NUM_SAMPLES,SC_9_WLR,SC_9_NUM_SAMPLES,SC_10_WLR,SC_10_NUM_SAMPLES,SC_11_WLR,SC_11_NUM_SAMPLES,SC_12_WLR,SC_12_NUM_SAMPLES,SC_13_WLR,SC_13_NUM_SAMPLES,SC_14_WLR,SC_14_NUM_SAMPLES,SC_15_WLR,SC_15_NUM_SAMPLES,SC_16_WLR,SC_16_NUM_SAMPLES,SC_17_WLR,SC_17_NUM_SAMPLES,SC_18_WLR,SC_18_NUM_SAMPLES,SC_19_WLR,SC_19_NUM_SAMPLES,SC_20_WLR,SC_20_NUM_SAMPLES,SC_21_WLR,SC_21_NUM_SAMPLES,SC_22_WLR,SC_22_NUM_SAMPLES,SC_23_WLR,SC_23_NUM_SAMPLES,SC_24_WLR,SC_24_NUM_SAMPLES,SC_25_WLR,SC_25_NUM_SAMPLES,SC_26_WLR,SC_26_NUM_SAMPLES,SC_27_WLR,SC_27_NUM_SAMPLES,SC_28_WLR,SC_28_NUM_SAMPLES,SC_29_WLR,SC_29_NUM_SAMPLES,SC_30_WLR,SC_30_NUM_SAMPLES,SC_31_WLR,SC_31_NUM_SAMPLES,SC_32_WLR,SC_32_NUM_SAMPLES,SC_33_WLR,SC_33_NUM_SAMPLES,SC_34_WLR,SC_34_NUM_SAMPLES,SC_35_WLR,SC_35_NUM_SAMPLES,SC_36_WLR,SC_36_NUM_SAMPLES,SC_37_WLR,SC_37_NUM_SAMPLES,SC_38_WLR,SC_38_NUM_SAMPLES,SC_39_WLR,SC_39_NUM_SAMPLES,SC_40_WLR,SC_40_NUM_SAMPLES,SC_41_WLR,SC_41_NUM_SAMPLES,SC_42_WLR,SC_42_NUM_SAMPLES,SC_43_WLR,SC_43_NUM_SAMPLES,SC_44_WLR,SC_44_NUM_SAMPLES,SC_45_WLR,SC_45_NUM_SAMPLES,SC_46_WLR,SC_46_NUM_SAMPLES,SC_47_WLR,SC_47_NUM_SAMPLES,SC_48_WLR,SC_48_NUM_SAMPLES,SC_49_WLR,SC_49_NUM_SAMPLES,SC_50_WLR,SC_50_NUM_SAMPLES,SC_51_WLR,SC_51_NUM_SAMPLES,SC_52_WLR,SC_52_NUM_SAMPLES,SC_53_WLR,SC_53_NUM_SAMPLES,SC_54_WLR,SC_54_NUM_SAMPLES,SC_55_WLR,SC_55_NUM_SAMPLES,SC_56_WLR,SC_56_NUM_SAMPLES,SC_57_WLR,SC_57_NUM_SAMPLES,SC_58_WLR,SC_58_NUM_SAMPLES,SC_59_WLR,SC_59_NUM_SAMPLES,SC_60_WLR,SC_60_NUM_SAMPLES,SC_61_WLR,SC_61_NUM_SAMPLES,SC_62_WLR,SC_62_NUM_SAMPLES,SC_63_WLR,SC_63_NUM_SAMPLES,SC_64_WLR,SC_64_NUM_SAMPLES,SC_65_WLR,SC_65_NUM_SAMPLES,SC_66_WLR,SC_66_NUM_SAMPLES,SC_67_WLR,SC_67_NUM_SAMPLES,SC_68_WLR,SC_68_NUM_SAMPLES,SC_69_WLR,SC_69_NUM_SAMPLES,SC_70_WLR,SC_70_NUM_SAMPLES,TOTAL_WLR,WLR_RANKING,SC_1_RANKING,SC_2_RANKING,SC_3_RANKING,SC_4_RANKING,SC_5_RANKING,SC_6_RANKING,SC_7_RANKING,SC_8_RANKING,SC_9_RANKING,SC_10_RANKING,SC_11_RANKING,SC_12_RANKING,SC_13_RANKING,SC_14_RANKING,SC_15_RANKING,SC_16_RANKING,SC_17_RANKING,SC_18_RANKING,SC_19_RANKING,SC_20_RANKING,SC_21_RANKING,SC_22_RANKING,SC_23_RANKING,SC_24_RANKING,SC_25_RANKING,SC_26_RANKING,SC_27_RANKING,SC_28_RANKING,SC_29_RANKING,SC_30_RANKING,SC_31_RANKING,SC_32_RANKING,SC_33_RANKING,SC_34_RANKING,SC_35_RANKING,SC_36_RANKING,SC_37_RANKING,SC_38_RANKING,SC_39_RA

In [27]:
df.drop(df.iloc[:, 111:544], inplace = True, axis = 1) 

# Remove certain columns

In [28]:
cols = sorted(df.columns.tolist())

In [29]:
# Remove due to dtype == object
cols.remove('BRED')
cols.remove('COLOURS_DESCRIPTION')
cols.remove('COLOUR_TYPE')
cols.remove('DAMSIRE_BRED')
cols.remove('DAMSIRE_NAME')
cols.remove('DAM_BRED')
cols.remove('DAM_NAME')
cols.remove('HORSE_FORM')
cols.remove('HORSE_RESULT')
cols.remove('JOCKEY_NAME')
cols.remove('LASTMATCHTIME')
cols.remove('LAST_RACE')
cols.remove('LAST_TWO_RACES')
cols.remove('LIVE_RACE_STATUS')
cols.remove('OWNER_NAME')
cols.remove('PROD_OF_NUM_SAMPLES_RANKINGS')
cols.remove('RACE_NAME')
cols.remove('RACE_START_TIME')
cols.remove('RACE_TRACK')
cols.remove('RACE_TYPE')
cols.remove('RUNNER_NAME')
cols.remove('RUNNER_STATUS')
cols.remove('SECOND_LAST_RACE')
cols.remove('SEX_TYPE')
cols.remove('SIRE_BRED')
cols.remove('SIRE_NAME')
cols.remove('TIME_OF_CHECK')
cols.remove('WEARING')
cols.remove('WEIGHT_UNITS')
cols.remove('TRAINER_NAME')

# Remove cos it's the label
cols.remove('WINNER')
cols.remove('LOSER')
cols.remove('REMOVED')
cols.remove('REWARD')

# Remove cos .....
cols.remove('SELECTIONID')
cols.remove('TOTALAVAILABLE')
cols.remove('Unnamed: 0')
cols.remove('SORT_PRIORITY')
cols.remove('DAMSIRE_YEAR_BORN')
cols.remove('RACE_ID')
cols.remove('MEETING_ID')
cols.remove('SIRE_YEAR_BORN')
cols.remove('PLACE_MARKETID')
cols.remove('MARKETID')
cols.remove('DAM_YEAR_BORN')
#cols.remove('SELECTIONID')

In [30]:
df[cols].dtypes

ADJUSTED_RATING                  float64
AGE                              float64
AGE_NORM                         float64
AMT_BACKED_10_MINS_OUT           float64
AMT_BACKED_10_MINS_OUT_NORM      float64
AMT_BACKED_1_MIN_OUT             float64
AMT_BACKED_1_MIN_OUT_NORM        float64
AMT_BACKED_20_MINS_OUT           float64
AMT_BACKED_20_MINS_OUT_NORM      float64
AMT_BACKED_2_SECS_OUT            float64
AMT_BACKED_2_SECS_OUT_NORM       float64
AMT_BACKED_30_MINS_OUT           float64
AMT_BACKED_30_MINS_OUT_NORM      float64
AMT_BACKED_40_MINS_OUT           float64
AMT_BACKED_40_MINS_OUT_NORM      float64
AMT_BACKED_45_MINS_OUT           float64
AMT_BACKED_45_MINS_OUT_NORM      float64
AMT_BACKED_5_MINS_OUT            float64
AMT_BACKED_5_MINS_OUT_NORM       float64
AMT_BACKED_60_MINS_OUT           float64
AMT_BACKED_60_MINS_OUT_NORM      float64
BACKPRICE                        float64
BACKPRICE_NORM                   float64
C                                  int64
CLOTH_NUMBER    

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [33]:
#X_train, X_test, y_train, y_test = train_test_split(df[cols], df.WINNER, test_size=0.2)

#now = time.time()

rfc_WINNER = RandomForestClassifier(n_estimators=100,n_jobs=-1,min_samples_split=12,random_state=42).fit(df[cols], df.WINNER)

#elapsed_time = int(time.time() - now)
#print (elapsed_time)



#result_A = X_test[[]].copy(True)

df['RF_Prob'] = rfc_WINNER.predict_proba(df[cols])[:,1]#*(3**0.35)

In [34]:
df.to_csv('HORSE_DATA_MOD.csv')

# Add in Ranked Features
## Group by Race then rank horses according to feature low to high.

In [ ]:
market_list = df.groupby('MARKETID').count().index.tolist()

feat_list = ['AMT_BACKED_1_MIN_OUT',\
             'AMT_BACKED_10_MINS_OUT',\
             'AMT_BACKED_20_MINS_OUT',\
             'AMT_BACKED_2_SECS_OUT',\
             'AMT_BACKED_30_MINS_OUT',\
             'AMT_BACKED_40_MINS_OUT',\
             'AMT_BACKED_45_MINS_OUT',\
             'AMT_BACKED_5_MINS_OUT',\
             'AMT_BACKED_60_MINS_OUT',\
             'BACKPRICE',\
             'AGE',
             'WEIGHT_VALUE',
             'JOCKEY_CLAIM',
             'PROPENSITY_RANKING',
             'HANDICAP']

for feat in feat_list:

    met_list = []
    for m_ID in market_list:
            
        
        df[df.MARKETID == m_ID].sort_values(by=[feat], ascending = False).reset_index()
        
        df[feat + '_RANKING']  = df.index.values



In [ ]:
df = df.sort_values(by=[feat], ascending = False)
df = df.reset_index()

df['rank_' + feat] = df.index.values
df['binned_' + feat] = pd.qcut(df['rank_' + feat],iles)
